Referências:  
http://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html  
https://numba.pydata.org/  
https://www.ibm.com/developerworks/community/blogs/jfp/entry/Python_Meets_Julia_Micro_Performance?lang=en  
https://gist.github.com/jfpuget/b53f1e15a37aba5944ad  

In [1]:
import random

Criando uma função Fibonacci Recursiva:

In [3]:
def fib(n):
    if n<2:
        return n
    return fib(n-1)+fib(n-2)

In [3]:
%timeit fib(20)
%timeit fib(30)

4.07 ms ± 550 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
553 ms ± 96.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Compilando a função com o compilador Cython:  
(pip3 install Cython)

In [5]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [9]:
%%cython

def fib_cython(n):
    if n<2:
        return n
    return fib_cython(n-1)+fib_cython(n-2)

In [10]:
%timeit fib_cython(20)
%timeit fib_cython(30)

KeyboardInterrupt: 

Podemos melhorar ainda mais compilando as variáveis estaticamente. A função deve ser declarada com "cpdef" ao invés de def". Dessa forma, podemos usar os tipos do C como parâmetros da função:

In [7]:
%%cython

cpdef long fib_cython_type(long n):
    if n<2:
        return n
    return fib_cython_type(n-1)+fib_cython_type(n-2)

In [8]:
%timeit fib_cython_type(20)
%timeit fib_cython_type(30)

63.7 µs ± 9.22 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
6.56 ms ± 1.08 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


Uma outra abordagem é permitindo que os resultados sejam guardados em _cache_:
Dessa forma podemos manter a precisão arbitrária das variáveis do Python, sem recorrer aos tipos estáticos do C

In [9]:
from functools import lru_cache as cache #Python3

@cache(maxsize=None)
def fib_cache(n):
    if n<2:
        return n
    return fib_cache(n-1)+fib_cache(n-2)

In [10]:
%timeit fib_cache(20)
%timeit fib_cache(30)

185 ns ± 32.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
158 ns ± 11.9 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Uma outra maneira de emular o cache seria realizando uma modificação no algoritmo:

In [11]:
def fib_seq(n):
    if n < 2:
        return n
    a,b = 1,0
    for i in range(n-1):
        a,b = a+b,a
    return a

In [12]:
%timeit fib_seq(20)
%timeit fib_seq(30)

1.95 µs ± 169 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
2.65 µs ± 446 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Vamos compilar esta nova função com e sem a utilização de tipagem estática: 

In [13]:
%%cython

def fib_seq_cython(n):
    if n < 2:
        return n
    a,b = 1,0
    for i in range(n-1):
        a,b = a+b,a
    return a

In [14]:
%timeit fib_seq_cython(20)
%timeit fib_seq_cython(30)

1.02 µs ± 93.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.28 µs ± 93.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [15]:
%%cython

cpdef long fib_seq_cython_type(long n):
    if n < 2:
        return n
    cdef long a,b
    a,b = 1,0
    for i in range(n-1):
        a,b = a+b,a
    return a

In [16]:
%timeit fib_seq_cython_type(20)
%timeit fib_seq_cython_type(30)

116 ns ± 12.9 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
122 ns ± 11.4 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Podemos usar uma ferramenta chamada Numba, que é um just-in-time (jit) compiler:

In [17]:
from numba import jit

In [18]:
@jit
def fib_seq_numba(n):
    if n < 2:
        return n
    a,b = 1,0
    for i in range(n-1):
        a,b = a+b,a
    return a

In [19]:
%timeit fib_seq_numba(20)
%timeit fib_seq_numba(30)

374 ns ± 71.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
369 ns ± 56.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Segundo exemplo:  
Algoritmo de ordenação Quick Sort  

In [20]:
def qsort_kernel(a, lo, hi):
    i = lo
    j = hi
    while i < hi:
        pivot = a[(lo+hi) // 2]
        while i <= j:
            while a[i] < pivot:
                i += 1
            while a[j] > pivot:
                j -= 1
            if i <= j:
                a[i], a[j] = a[j], a[i]
                i += 1
                j -= 1
        if lo < j:
            qsort_kernel(a, lo, j)
        lo = i
        j = hi
    return a

def benchmark_qsort():
    lst = [ random.random() for i in range(1,5000) ]
    qsort_kernel(lst, 0, len(lst)-1)

Usando Numba:

In [21]:
@jit
def qsort_kernel_numba(a, lo, hi):
    i = lo
    j = hi
    while i < hi:
        pivot = a[(lo+hi) // 2]
        while i <= j:
            while a[i] < pivot:
                i += 1
            while a[j] > pivot:
                j -= 1
            if i <= j:
                a[i], a[j] = a[j], a[i]
                i += 1
                j -= 1
        if lo < j:
            qsort_kernel_numba(a, lo, j)
        lo = i
        j = hi
    return a

def benchmark_qsort_numba():
        lst = [ random.random() for i in range(1,5000) ]
        qsort_kernel_numba(lst, 0, len(lst)-1)

Numba e Numpy:

In [22]:
@jit
def qsort_kernel_numba_numpy(a, lo, hi):
    i = lo
    j = hi
    while i < hi:
        pivot = a[(lo+hi) // 2]
        while i <= j:
            while a[i] < pivot:
                i += 1
            while a[j] > pivot:
                j -= 1
            if i <= j:
                a[i], a[j] = a[j], a[i]
                i += 1
                j -= 1
        if lo < j:
            qsort_kernel_numba_numpy(a, lo, j)
        lo = i
        j = hi
    return a

def benchmark_qsort_numba_numpy():
        lst = np.random.rand(5000)
        qsort_kernel_numba(lst, 0, len(lst)-1)

Usando Numpy e Cython:  

In [23]:
%%cython
import numpy as np
import cython

@cython.boundscheck(False)
@cython.wraparound(False)
cdef double[:] \
qsort_kernel_cython_numpy_type(double[:] a, \
                               long lo, \
                               long hi):
    cdef: 
        long i, j
        double pivot
    i = lo
    j = hi
    while i < hi:
        pivot = a[(lo+hi) // 2]
        while i <= j:
            while a[i] < pivot:
                i += 1
            while a[j] > pivot:
                j -= 1
            if i <= j:
                a[i], a[j] = a[j], a[i]
                i += 1
                j -= 1
        if lo < j:
            qsort_kernel_cython_numpy_type(a, lo, j)
        lo = i
        j = hi
    return a

def benchmark_qsort_numpy_cython():
    lst = np.random.rand(5000)
    qsort_kernel_cython_numpy_type(lst, 0, len(lst)-1)

def benchmark_sort_numpy():
    lst = np.random.rand(5000)
    np.sort(lst)

In [24]:
%timeit benchmark_qsort()
%timeit benchmark_qsort_numba()
%timeit benchmark_qsort_numba_numpy()
%timeit benchmark_qsort_numpy_cython()
%timeit benchmark_sort_numpy()

23.3 ms ± 4.13 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
1.6 ms ± 97.3 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
689 µs ± 86.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
942 µs ± 163 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
570 µs ± 97 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
